## 🏷️ Funzionamento dello Script (Passo dopo Passo)

### 1) Caricamento e Pre-elaborazione dei Dati
Si parte caricando il file CSV contenente le informazioni aziendali. Dopo aver letto il dataset, vengono selezionate le sole colonne considerate importanti per il matching, in questo caso:

- `company_name`
- `industry`
- `headquarters_city`

A questo punto, si eliminano (drop) le righe che non hanno il nome dell’azienda (`company_name`), perché tale colonna è ritenuta fondamentale per confrontare i record. Infine, si stampa quanti record validi rimangono dopo questa pulizia.

**Perché si fa:** riduciamo il DataFrame a ciò che serve per identificare la stessa entità, eliminando dati mancanti sui campi essenziali.

### 2) Creazione di un “Blocking” non troppo restrittivo
Lo script utilizza la libreria `recordlinkage` per definire le regole di blocco (blocking). In particolare, si creano due regole:

- **Exact Block su `industry`:**
    Raggruppa i record per settore, così il confronto avviene solo tra aziende che appartengono allo stesso `industry`.

- **SortedNeighbourhood su `company_name` con finestra = 5:**
    Ordina le aziende in base al nome e poi confronta ogni azienda con le 5 precedenti e le 5 successive. Lo scopo è catturare nomi che sono “vicini” a livello alfabetico, evitando di confrontare aziende i cui nomi sarebbero nettamente diversi.

Il risultato di queste due regole viene unito (logica OR). Ciò significa che una coppia di record passa alla fase successiva se soddisfa almeno una delle due condizioni di blocco.

Alla fine, si stampa quante coppie candidate abbiamo ottenuto: è il numero di possibili confronti che andranno analizzati più a fondo.

**Perché si fa:** il blocking è usato per ridurre il numero di confronti dal potenziale \(O(n^2)\) a un insieme più piccolo e verosimile. Si sceglie un blocco “non troppo restrittivo” così da non perdere troppi possibili match.

### 3) Definizione delle Regole di Confronto (Compare)
Si istanzia un oggetto `Compare()` di `recordlinkage`, in cui definiamo le metriche usate per confrontare le coppie candidate:

- **Somiglianza di stringa (Jaro-Winkler) per `company_name`:**
    Calcola un punteggio da 0 (completamente diversi) a 1 (identici). Jaro-Winkler è utile per correggere piccoli errori di battitura o variazioni del nome.

- **Exact Match per `industry` e `headquarters_city`:**
    Se i due record hanno esattamente lo stesso valore, la metrica vale 1, altrimenti 0.

In sostanza, così facendo, potremo valutare se i nomi delle aziende sono molto simili, e allo stesso tempo se `industry` e `headquarters_city` coincidono.

### 4) Calcolo della Matrice di Similarità
Viene calcolata la `similarity_matrix` per tutte le coppie candidate prodotte dal blocking. Ogni riga della matrice corrisponde a una coppia \((record_A, record_B)\); ogni colonna è una metrica (es. `name_sim`, `industry_exact`, `city_exact`). Si stampa la dimensione di questa matrice, cioè quante coppie e quante metriche sono state calcolate.

**Perché si fa:** per ogni coppia ritenuta potenzialmente “sospetta” (stessa `industry` o nome vicino), si ottengono punteggi che ci aiutano a decidere se la coppia è davvero la stessa azienda.

### 5) Selezione delle Coppie Considerate ‘Match’
Dalla matrice di similarità, si estraggono solo le righe che soddisfano certe regole:

- Nome simile (`name_sim > 0.95`), oppure
- Stesso settore e stessa città (`industry_exact == 1` e `city_exact == 1`).

Le coppie che rispettano almeno una di queste condizioni vengono prese come “match”. Si stampa quante coppie totali soddisfano i criteri.

**Perché si fa:** abbiamo impostato delle soglie (es. 0.95 su Jaro-Winkler) e regole di abbinamento. A seconda dei dati, si può decidere di alzare/abbassare le soglie per ottenere più o meno match.

### 6) Recupero dei Valori Originali tramite “Join”
Dopo aver deciso quali coppie sono “match”, si vuole visualizzare i dati originali (il nome, il settore, la città per entrambi i record). Per farlo:

- Si resetta l’indice (che era una `MultiIndex` di coppie) per ottenere due colonne: `id_left` e `id_right`, corrispondenti alle posizioni dei record nel DataFrame originale.
- Si fa un merge con `df` usando `id_left` e poi un altro merge con `id_right`. Così si recuperano i valori effettivi delle colonne `company_name`, `industry`, `headquarters_city` per entrambi i record della coppia (quelli “di sinistra” e “di destra”).

Ora nel DataFrame finale troviamo colonne come: `company_name_left`, `company_name_right`, `industry_left`, `industry_right`, e così via, affiancate ai punteggi di similarità.

**Perché si fa:** la tabella di similarità di base contiene solo i punteggi e gli indici delle coppie. Per controllare “a occhio” se i match hanno senso, serve ricostruire i valori testuali originali.

### 7) Salvataggio del Risultato in CSV
Infine, si esporta il DataFrame risultante (che contiene le coppie abbinate più le colonne left e right) in un file CSV, in modo da avere un report di tutti i match, con i punteggi di somiglianza e i dati affiancati.

**Perché si fa:** questo file finale consente di verificare manualmente (o mostrare al professore/ai colleghi) i match individuati, con la prova testuale delle colonne corrispondenti.

In [38]:
import pandas as pd
import recordlinkage

# 1) Caricamento e pre-elaborazione dati
schema_file = "main_outputs/final_mediated_schema.csv"
df = pd.read_csv(schema_file)

# Seleziona alcune colonne di esempio
df = df[['company_name', 'industry', 'headquarters_city']]
df.dropna(subset=['company_name'], inplace=True)

print(f"Numero di record iniziali: {len(df)}")

# 2) Creazione di un blocking non troppo restrittivo
indexer = recordlinkage.Index()
indexer.add(recordlinkage.index.Block('industry'))
indexer.add(recordlinkage.index.SortedNeighbourhood('company_name', window=5))

candidate_pairs = indexer.index(df)
print(f"Numero di coppie candidate: {len(candidate_pairs)}")

# 3) Definizione delle regole di confronto
compare = recordlinkage.Compare()

compare.string('company_name', 'company_name', method='jarowinkler', label='name_sim')
compare.exact('industry', 'industry', label='industry_exact')
compare.exact('headquarters_city', 'headquarters_city', label='city_exact')

# 4) Calcolo della matrice di similarità
similarity_matrix = compare.compute(candidate_pairs, df)
print(f"Dimensioni della similarity_matrix: {similarity_matrix.shape}")

# 5) Selezione coppie considerate match (esempio di regola)
matches = similarity_matrix[
    (similarity_matrix['name_sim'] > 0.8) &  (similarity_matrix['name_sim'] < 0.95) |
    ((similarity_matrix['name_sim'] > 0.9) & (similarity_matrix['industry_exact'] == 1) &
     (similarity_matrix['city_exact'] == 1))
]

print(f"Numero di coppie finali considerate 'match': {len(matches)}")

# 6) Per avere i valori effettivi dei record, facciamo un "join" con df originale

# a) Resettiamo l'indice (che è una MultiIndex di coppie) per avere due colonne: 'level_0' e 'level_1'
matches = matches.reset_index()
matches.rename(columns={'level_0':'id_left','level_1':'id_right'}, inplace=True)

# Ora 'id_left' e 'id_right' sono gli indici del DataFrame df

# b) Uniamo i valori delle colonne originali di 'df' per id_left
matches = matches.merge(df, left_on='id_left', right_index=True, how='left', suffixes=('', '_left'))

# c) Uniamo i valori delle colonne originali di 'df' per id_right
matches = matches.merge(df, left_on='id_right', right_index=True, how='left', suffixes=('_left', '_right'))

# Adesso, matches ha le colonne del record "di sinistra" (id_left) e del record "di destra" (id_right)
# Es. company_name_left, industry_left, headquarters_city_left, company_name_right, industry_right, etc.
# E contiene anche le colonne di similarità: name_sim, industry_exact, city_exact

# INFINE, eliminiamo da matches, tutte le righe in cui i campi di company_name hanno esattamente la stessa identica stringa
matches = matches[matches['company_name_left'] != matches['company_name_right']]

# 7) Salvataggio in CSV con tutti i dati
matches.to_csv("matched_companies_detailed.csv", index=False)
print("✅ File 'matched_companies_detailed.csv' generato con i valori dei record affiancati!")


C:\Users\luca-\AppData\Local\Temp\ipykernel_21184\3244916458.py:6: DtypeWarning: Columns (2,5,6,7,9,14,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(schema_file)


Numero di record iniziali: 75793
Numero di coppie candidate: 7358685
Dimensioni della similarity_matrix: (7358685, 3)
Numero di coppie finali considerate 'match': 112043
✅ File 'matched_companies_detailed.csv' generato con i valori dei record affiancati!


## Cosa contiene il file finale

Dopo questo passaggio, il CSV `matched_companies_detailed.csv` avrà le colonne:

- `id_left` e `id_right`: gli indici Pandas dei due record matchati.
- `name_sim`, `industry_exact`, `city_exact`: le metriche di somiglianza calcolate.
- `company_name_left`, `industry_left`, `headquarters_city_left`: i valori presi dal DataFrame originale per il record "di sinistra".
- `company_name_right`, `industry_right`, `headquarters_city_right`: idem per il record "di destra".

In questo modo, visivamente puoi ispezionare se `company_name_left` e `company_name_right` sembrano davvero la stessa azienda, controllare se `industry_left` = `industry_right`, e così via.

---

## Script completo che esegue il campionamento del CSV `matched_companies_detailed.csv` seguendo i criteri indicati

### Passaggi:

1. Filtra le coppie con similarità (`name_sim`) tra 0.6 e 0.7 (o intervallo che preferisci).
2. Opzionalmente filtra per diversità (ad es. `industry_left` != `industry_right` o altre condizioni).
3. Applica un campionamento sistematico per ottenere esattamente 2000 coppie.



In [43]:
import pandas as pd
from difflib import SequenceMatcher

# Leggi il CSV originale
df = pd.read_csv("matched_companies_detailed.csv")

# Normalizza i nomi delle aziende
df["company_name_left"] = df["company_name_left"].str.strip().str.lower()
df["company_name_right"] = df["company_name_right"].str.strip().str.lower()

# Funzione di similarità
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Calcoliamo la similarità su ogni riga
df["name_similarity"] = df.apply(lambda row: similar(row["company_name_left"], row["company_name_right"]), axis=1)

# Filtra per similarità (tra 0.6 e 0.7 come nella ground truth)
filtered_df = df[(df["name_similarity"] >= 0.6) & (df["name_similarity"] <= 0.7)]

# Conta le righe rimaste
num_rows = len(filtered_df)

# Calcola il passo per ottenere ~2000 righe
step = max(1, num_rows // 1000)

# Campionamento sistematico ogni 'step' righe
sampled_df = filtered_df.iloc[::step]

# Seleziona solo le prime 2000 righe
sampled_df = sampled_df.head(1000)

# Salva il CSV campionato
sampled_df.to_csv("matched_companies_sampled.csv", index=False)

print(f"Campionamento completato: {len(sampled_df)} righe salvate in 'matched_companies_sampled.csv'")


Campionamento completato: 1000 righe salvate in 'matched_companies_sampled.csv'


## 📌 Passaggio 1: Convertire ground_truth.json in CSV

Attualmente il file JSON ha questa struttura:

```json
{
    "ground_truth": [
        {
            "1&1": "companiesMarketCap_dataset",
            "1&1 AG": "disfold-com"
        },
        {
            "AMPLUS": "ft-com",
            "AMPLUS SOLAR": "AmbitionBox"
        }
    ]
}
```

Dobbiamo convertirlo in un formato compatibile con il CSV che abbiamo creato. Un buon formato potrebbe essere:

```csv
company_a,source_a,company_b,source_b
1&1,companiesMarketCap_dataset,1&1 AG,disfold-com
AMPLUS,ft-com,AMPLUS SOLAR,AmbitionBox
```

In [6]:
import json
import csv

# Leggiamo il JSON
with open("data/ground_truth.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# data["ground_truth"] contiene una lista di oggetti
# Ognuno di questi oggetti ha esattamente 2 coppie "nome": "fonte", es:
# {
#   "1&1": "companiesMarketCap_dataset",
#   "1&1 AG": "disfold-com"
# }

# Creiamo il file CSV in scrittura
with open("data/ground_truth.csv", "w", encoding="utf-8", newline="") as csv_file:
    writer = csv.writer(csv_file)
    # Scriviamo l'header
    writer.writerow(["company_a", "source_a", "company_b", "source_b"])

    # Per ogni oggetto in data["ground_truth"], estraiamo i 2 pair
    for pair_dict in data["ground_truth"]:
        # pair_dict è un dizionario, es: {"1&1": "companiesMarketCap_dataset", "1&1 AG": "disfold-com"}
        items = list(pair_dict.items())  # Convertiamo in lista di tuple [(company, source), (company, source)]
        
        # Assumiamo che ci siano esattamente 2 coppie
        (company_a, source_a) = items[0]
        (company_b, source_b) = items[1]

        # Scriviamo una riga nel CSV
        writer.writerow([company_a, source_a, company_b, source_b])

print("Conversione completata! File 'ground_truth.csv' generato.")


Conversione completata! File 'ground_truth.csv' generato.


```markdown
---

## 📊 Valutazione dei Risultati Ottenuti

In questa sezione, analizzeremo i risultati ottenuti dal processo di matching delle aziende. Valuteremo l'accuratezza e la qualità dei match identificati, confrontandoli con il ground truth disponibile.

---
```

In [44]:
import pandas as pd
from recordlinkage.measures import precision, recall, fscore

# Ground truth
gt_df = pd.read_csv("data/ground_truth.csv")
gt_df["company_a"] = gt_df["company_a"].str.strip().str.lower()
gt_df["company_b"] = gt_df["company_b"].str.strip().str.lower()

true_pairs = {
    tuple(sorted([row["company_a"], row["company_b"]]))
    for _, row in gt_df.iterrows()
}
# Convertiamo in MultiIndex
true_links = pd.MultiIndex.from_tuples(true_pairs, names=["company_left", "company_right"])

# Predetti
pred_df = pd.read_csv("matched_companies_sampled.csv")
pred_df["company_name_left"] = pred_df["company_name_left"].str.strip().str.lower()
pred_df["company_name_right"] = pred_df["company_name_right"].str.strip().str.lower()

pred_pairs = {
    tuple(sorted([row["company_name_left"], row["company_name_right"]]))
    for _, row in pred_df.iterrows()
}
# Convertiamo in MultiIndex
pred_links = pd.MultiIndex.from_tuples(pred_pairs, names=["company_left", "company_right"])

# Calcolo metriche
p = precision(true_links, pred_links)
r = recall(true_links, pred_links)

print("=== RISULTATI RECORD LINKAGE (names, MultiIndex) ===")
print(f"Precision: {p:.3f}")
print(f"Recall:    {r:.3f}")


=== RISULTATI RECORD LINKAGE (names, MultiIndex) ===
Precision: 0.012
Recall:    0.059
